In [1]:
import os

In [2]:
os.getcwd()

'/Users/mac/Desktop/Kidney-Disease_Classifier--DeepLearning Project/research'

In [3]:
os.getcwd()

'/Users/mac/Desktop/Kidney-Disease_Classifier--DeepLearning Project/research'

In [4]:
os.chdir("../")

In [5]:
os.getcwd()

'/Users/mac/Desktop/Kidney-Disease_Classifier--DeepLearning Project'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class BaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [11]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])



    def get_base_model_config(self) -> BaseModelConfig:
        config = self.config.base_model

        create_directories([config.root_dir])

        base_model_config = BaseModelConfig(
            root_dir=Path(config.root_dir), 
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path), 
            params_image_size=self.params.IMAGE_SIZE, 
            params_learning_rate=self.params.LEARNING_RATE, 
            params_include_top=self.params.INCLUDE_TOP, 
            params_weights=self.params.WEIGHTS, 
            params_classes=self.params.CLASSES
        )

        return base_model_config

In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

2025-03-16 22:10:31.801258: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
class BaseModel:
    def __init__(self, config: BaseModelConfig):
        self.config = config


    def get_base_model(self):
        self.model = tf.keras.applications.ResNet101V2(
            input_shape=self.config.params_image_size, 
            weights=self.config.params_weights, 
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)



    @staticmethod
    def prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes, 
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input, 
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.AdamW(learning_rate=learning_rate), 
            loss=tf.keras.losses.CategoricalCrossentropy(), 
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model
    
    def update_base_model(self):
        self.full_model = self.prepare_full_model(
            model=self.model, 
            classes=self.config.params_classes, 
            freeze_all=True, 
            freeze_till=None, 
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)
    
    

    @staticmethod
    def save_model(path: Path, model:tf.keras.Model):
        model.save(path)

In [13]:
try:
    config = ConfigurationManager()
    base_model_config = config.get_base_model_config()
    base_model = BaseModel(config=base_model_config)
    base_model.get_base_model()
    base_model.update_base_model()
except Exception as e:
    raise e

[2025-03-16 23:26:30,697: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-16 23:26:30,710: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-16 23:26:30,716: INFO: common: created directory at: artifacts]
[2025-03-16 23:26:30,718: INFO: common: created directory at: artifacts/base_model]
171317808/171317808 [==============================] - 46s 0us/step
[2025-03-16 23:27:24,918: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                     